<a href="https://colab.research.google.com/github/luishenriquert/luishenriquert/blob/main/Training_Interactive_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import folium
import pandas as pd
import numpy as np
from folium.plugins import HeatMap
import plotly.graph_objs as go
from folium.plugins import MarkerCluster

# Function to classifier clinical evolution
#Here I classified in blood glucose (glicemia) and blood count (hemograma)
def classificar_evolucao(glicemia, hemograma):
    if glicemia > 7.0 or hemograma < 4.5:
        return 'Piora'
    elif glicemia <= 6.0 and hemograma >= 4.5:
        return 'Evolução'
    else:
        return 'Estável'

# Generating random data
n_pacientes = 100000
nomes = [f'Paciente_{i}' for i in range(1, n_pacientes + 1)]
ceps_anapolis = ['75000-000', '75010-000', '75020-000', '75030-000', '75040-000', '75050-000', '75060-000']
ceps = np.random.choice(ceps_anapolis, size=n_pacientes)
glicemia = np.random.uniform(4.0, 12.0, size=n_pacientes)  # Example of blood glucose over the last 3 years
hemograma = np.random.uniform(3.0, 6.0, size=n_pacientes)  # Example of blood count from the last 3 years

# Creat DataFrame
df = pd.DataFrame({
    'Nome': nomes,
    'CEP': ceps,
    'Glicemia': glicemia,
    'Hemograma': hemograma
})

# Classify patients based on blood glucose and blood count
df['Status'] = df.apply(lambda x: classificar_evolucao(x['Glicemia'], x['Hemograma']), axis=1)

# Group data by zip code and status
agrupado_por_cep = df.groupby(['CEP', 'Status']).size().unstack(fill_value=0)

# Base Map
mapa = folium.Map(location=[-16.3285, -48.9534], zoom_start=12)

# Coordinates for each ZIP code
coordenadas_cep = {
    '75000-000': [-16.3285, -48.9534],
    '75010-000': [-16.3211, -48.9490],
    '75020-000': [-16.3223, -48.9400],
    '75030-000': [-16.3350, -48.9550],
    '75040-000': [-16.3412, -48.9600],
    '75050-000': [-16.3250, -48.9300],
    '75060-000': [-16.3100, -48.9650]
}

# Add patient clusters to the map
marker_cluster = MarkerCluster().add_to(mapa)

# Add markers to the map with progress/pain information
for cep, row in agrupado_por_cep.iterrows():
    lat, lon = coordenadas_cep[cep]
    popup_text = (f"CEP: {cep}<br>"
                  f"Pacientes com Piora: {row['Piora']}<br>"
                  f"Pacientes com Evolução: {row['Evolução']}")
    folium.Marker(
        location=[lat, lon],
        popup=popup_text
    ).add_to(marker_cluster)

# Convert values from int64 to int when creating heat_data
heat_data_piora = [[coordenadas_cep[cep][0], coordenadas_cep[cep][1], int(row['Piora'])] for cep, row in agrupado_por_cep.iterrows()]
heat_data_evolucao = [[coordenadas_cep[cep][0], coordenadas_cep[cep][1], int(row['Evolução'])] for cep, row in agrupado_por_cep.iterrows()]

# Add HeatMap with adjusted data
HeatMap(heat_data_piora, name='Pacientes com Piora', radius=15, max_zoom=13).add_to(mapa)
HeatMap(heat_data_evolucao, name='Pacientes com Evolução', radius=15, max_zoom=13, gradient={0.4: 'green', 1: 'blue'}).add_to(mapa)

# Save the map in an HTML file
mapa.save("mapa_evolucao_piora.html")

# Display the interactive map directly in Colab
mapa